# Regression Week 5: Ass 1 : Feature Selection and LASSO (Interpretation)
###graphlab version  see also https://rpubs.com/Dezwirey/190659  and https://github.com/anindya-saha/Machine-Learning-with-Python/blob/master/Coursera-Machine-Learning-Regression/week-5-lasso-assignment-1-graphlab.ipynb

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create
**0.** Load the sales dataset:

In [1]:
import graphlab as gl
import math
import random
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = gl.SFrame('../Week2/kc_house_data.gl/')
sales.head(1)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1472181351.log


This non-commercial license of GraphLab Create for academic use is assigned to vinorda@gmail.com and will expire on June 04, 2017.


Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 1

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|       0       |   1955   |      0       |  98178  | 47.51123398 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
+---------------+---------------+-----+
[1 rows x 21 columns]

# Create new features

**1.** Create new features by performing following transformation on inputs:
As in Week 2, we consider features that are some transformations of inputs. 

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']
print sales.column_names()

['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt', 'sqft_lot_sqrt', 'bedrooms_square', 'floors_square']


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

**2.** Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.
Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = gl.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0.0, l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.763957     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.814885     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
print('number of nonzeros = %d' % (model_all.coefficients['value']).nnz())
print "Features that had non-zero weight\n", model_all.coefficients[model_all.coefficients['value'] != 0]
print(model_all.coefficients.print_rows(num_rows=model_all.num_coefficients))

number of nonzeros = 6
Features that had non-zero weight
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
+------------------+-------+---------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |    

In [7]:
model_all.coefficients[model_all.coefficients['value'] == 0]

name,index,value,stderr
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None
waterfront,None,0.0,None
view,None,0.0,None
condition,None,0.0,None
sqft_basement,None,0.0,None


In [8]:
model_all.coefficients[model_all.coefficients['value'] > 0]['name']

dtype: str
Rows: ?
['(intercept)', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ]

Note that a majority of the weights have been set to zero. So by **setting a very large L1 penalty**, we are performing a **subset or feature selection**. 

**3. QUIZ QUESTION Q1: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?**
According to this list of weights, which of the features have been chosen?  'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above'

 
Which of the following features have been chosen by LASSO, i.e. which features were assigned nonzero weights? (Choose all that apply)

* yr_renovated
* waterfront
* sqft_living   <----
* grade         <----
* floors

# Selecting an L1 penalty

**4.** To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

**5.** Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [10]:
penalty_rss = []
min_rss = None
best_l1_penalty = None
best_model = None
for l1_penalty in np.logspace(1, 7, num=13):
    # First fit model to training data
    model = gl.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty)
    # Then get the predictions
    predictions = model.predict(validation[all_features])
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = validation['price'] - predictions
    # square the residuals and add them up
    rss = (residuals**2).sum()
    penalty_rss.append((l1_penalty, rss))
    
    print "l1_penalty: %0.2f, RSS: $%0.2f, Num of non-zero coefficients:%d" \
          %(l1_penalty, rss, len(model.coefficients[model.coefficients['value'] != 0]))    
    if min_rss is None or rss < min_rss:
        min_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model

        
print "\n\nOverall Best l1_penalty on validation data = %0.2f, as it gives lowest rss of %0.2f" %(best_l1_penalty,min_rss)    

l1_penalty: 10.00, RSS: $625766285142460.50, Num of non-zero coefficients:18
l1_penalty: 31.62, RSS: $625766285362394.38, Num of non-zero coefficients:18
l1_penalty: 100.00, RSS: $625766286057885.12, Num of non-zero coefficients:18
l1_penalty: 316.23, RSS: $625766288257224.38, Num of non-zero coefficients:18
l1_penalty: 1000.00, RSS: $625766295212186.12, Num of non-zero coefficients:18
l1_penalty: 3162.28, RSS: $625766317206080.75, Num of non-zero coefficients:18
l1_penalty: 10000.00, RSS: $625766386760658.00, Num of non-zero coefficients:18
l1_penalty: 31622.78, RSS: $625766606749278.50, Num of non-zero coefficients:18
l1_penalty: 100000.00, RSS: $625767302791634.88, Num of non-zero coefficients:18
l1_penalty: 316227.77, RSS: $625769507643885.75, Num of non-zero coefficients:18
l1_penalty: 1000000.00, RSS: $625776517727024.50, Num of non-zero coefficients:18
l1_penalty: 3162277.66, RSS: $625799062845466.62, Num of non-zero coefficients:18
l1_penalty: 10000000.00, RSS: $625883719085425

In [11]:
penalty_rss = sorted(penalty_rss, key = lambda x : x[1])
l1_penalty_best = penalty_rss[0][0]
print(l1_penalty_best)

print "The best model has", (best_model['coefficients']['value'].nnz()), "non zero coefficients, its RSS on validation is",\
min_rss, "for a lambda of", best_l1_penalty

10.0
The best model has 18 non zero coefficients, its RSS on validation is 6.25766285142e+14 for a lambda of 10.0


*** 6. QUIZ QUESTIONS Q2: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?***
1. What was the best value for the `l1_penalty`? 10.0
2. What is the RSS on TEST data of the model with the best `l1_penalty`? 1.56983602382e+14

In which of the following ranges does the best l1_penalty fall?
* Between 0 and 100    <----
* Between 100 and 1000
* Between 1000 and 10000
* Between 10000 and 100000
* Greater than 100000

**7.** Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty. 1.56983602382e+14


In [12]:
# No need to train again using best_l1 above as we captured model using best_l1 penality in loop earlier..
# so no need to do this
model_all = gl.linear_regression.create(training, target='price', features=all_features, validation_set=None,
                                                  l2_penalty=0., l1_penalty=l1_penalty_best, verbose=False)

# First get the predictions
predictions = best_model.predict(testing[all_features])
# then compute the residuals
residuals = testing['price'] - predictions
# square the residuals and add them up
rss_test = (residuals**2).sum()
print("RSS on TEST data: $%.6f" % (rss_test))
#RSS on TEST data: $156983602381664.187500

RSS on TEST data: $156983602381664.125000


**8. QUIZ QUESTION - Q3 Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero.**

Have 17 features so 17 coef plus 1 intercept - total 18, all 18 non-zero, NB sklearn.Lasso gives 15 non-zero only.

In [13]:
print len(best_model.coefficients[best_model.coefficients['value'] != 0])
print best_model.coefficients[best_model.coefficients['value'] != 0]. \
                                      print_rows(num_rows=20, num_columns=4)

print ('number of nonzeros = %d' %(best_model.coefficients['value'].nnz()))

18
+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |

# Limit the number of nonzero weights

**9.** What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

**10.** Assign 7 to the variable ‘max_nonzeros’.

In [14]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

**11.** Exploring large range of l1_penalty, so let's define a wide range of possible `l1_penalty_values`:

In [15]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [26]:
l1_penalty_values = np.logspace(8, 10, num=20)
max_nonzeros = 7
nonzero_list = []
l1_penalty_min = None
l1_penalty_max = None

for l1_penalty in l1_penalty_values:
    model = gl.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty)
    
    num_nonzero_coef = model['coefficients']['value'].nnz()
    nonzero_list.append( (l1_penalty, num_nonzero_coef) )
    # create a list of tuples

    print "\nl1_penalty = %0.2f: non-zero coefficients = %d" \
          %(l1_penalty, num_nonzero_coef)    
    
    # There is more efficient list comprehension means to grab this as well 
    # so not really need the following statements to find min and max!!
    
    if ( l1_penalty_min  is None) or (num_nonzero_coef > max_nonzeros):
        l1_penalty_min = l1_penalty
        nonzero_coef_max = num_nonzero_coef
    
    if (l1_penalty_max is None) and (num_nonzero_coef < max_nonzeros):
        l1_penalty_max = l1_penalty
        nonzero_coef_min = num_nonzero_coef
        
nonzero_list        


l1_penalty = 100000000.00: non-zero coefficients = 18

l1_penalty = 127427498.57: non-zero coefficients = 18

l1_penalty = 162377673.92: non-zero coefficients = 18

l1_penalty = 206913808.11: non-zero coefficients = 18

l1_penalty = 263665089.87: non-zero coefficients = 17

l1_penalty = 335981828.63: non-zero coefficients = 17

l1_penalty = 428133239.87: non-zero coefficients = 17

l1_penalty = 545559478.12: non-zero coefficients = 17

l1_penalty = 695192796.18: non-zero coefficients = 17

l1_penalty = 885866790.41: non-zero coefficients = 16

l1_penalty = 1128837891.68: non-zero coefficients = 15

l1_penalty = 1438449888.29: non-zero coefficients = 15

l1_penalty = 1832980710.83: non-zero coefficients = 13

l1_penalty = 2335721469.09: non-zero coefficients = 12

l1_penalty = 2976351441.63: non-zero coefficients = 10

l1_penalty = 3792690190.73: non-zero coefficients = 6

l1_penalty = 4832930238.57: non-zero coefficients = 5

l1_penalty = 6158482110.66: non-zero coefficients = 3

l1_p

[(100000000.0, 18),
 (127427498.57031322, 18),
 (162377673.91887242, 18),
 (206913808.11147901, 18),
 (263665089.87303555, 17),
 (335981828.62837881, 17),
 (428133239.8719396, 17),
 (545559478.11685145, 17),
 (695192796.17755914, 17),
 (885866790.41008317, 16),
 (1128837891.6846883, 15),
 (1438449888.2876658, 15),
 (1832980710.8324375, 13),
 (2335721469.0901213, 12),
 (2976351441.6313133, 10),
 (3792690190.7322536, 6),
 (4832930238.5717525, 5),
 (6158482110.6602545, 3),
 (7847599703.5146227, 1),
 (10000000000.0, 1)]

In [17]:
print "\nThe largest l1_penalty that has more non-zeros than %d is %0.2f with %d non-zero coefs" \
%(max_nonzeros, l1_penalty_min, nonzero_coef_max )    
print "\nThe smallest l1_penalty that has fewer non-zeros than %d is %0.2f with %d non-zero coefs" \
%(max_nonzeros, l1_penalty_max, nonzero_coef_min)   


The largest l1_penalty that has more non-zeros than 7 is 2976351441.63 with 10 non-zero coefs

The smallest l1_penalty that has fewer non-zeros than 7 is 3792690190.73 with 6 non-zero coefs


Thus Minimum lambda 2976351441.63    Maximum lambda 3792690190.73

**12.** Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above.... we did this above in the loop
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries - we do this below..

In [27]:
# just showing off list comprehension that also works and for loop will be much smaller above
# this uses list of tuple created in loop above
# `nonzero_list.append( (l1_penalty, num_nonzero_coef) )` 
l1_penalty_min = max([t[0] for t in nonzero_list if t[1] > max_nonzeros])
l1_penalty_max = min([t[0] for t in nonzero_list if t[1] < max_nonzeros])
print(l1_penalty_min, l1_penalty_max)

(2976351441.6313133, 3792690190.7322536)


In [30]:
print l1_penalty_min
print l1_penalty_max

2976351441.63
3792690190.73


***13. QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

**14.** Exploring narrower range/region of `l1_penalty` values we found:between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces **exactly the right number of nonzeros AND also minimizes RSS on the VALIDATION set**.

In [20]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

**15.** Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [21]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
penalty_rss_model = []
min_rss = None
best_l1_penalty = None
best_model = None

for l1_penalty in l1_penalty_values :
    model = gl.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False, 
                                              l2_penalty=0.0, l1_penalty=l1_penalty)
    
    model_nnz = model['coefficients']['value'].nnz()
    
    predictions = model.predict(validation[all_features])
    residuals = validation['price'] - predictions
    rss = (residuals**2).sum() 
      
    print "\nl1_penalty = %0.2f: non-zero coefficients = %d: rss= %0.2f" \
          %(l1_penalty,  model_nnz, rss)
        
    if (model_nnz == max_nonzeros) and (min_rss is None or rss < min_rss):
        min_rss = rss
        best_l1_penalty = l1_penalty
        best_model = model
        penalty_rss_model.append((l1_penalty, rss, model))


l1_penalty = 2976351441.63: non-zero coefficients = 10: rss= 966925692362085.25

l1_penalty = 3019316638.95: non-zero coefficients = 10: rss= 974019450084554.62

l1_penalty = 3062281836.27: non-zero coefficients = 10: rss= 981188367942453.75

l1_penalty = 3105247033.59: non-zero coefficients = 10: rss= 989328342459473.50

l1_penalty = 3148212230.92: non-zero coefficients = 10: rss= 998783211265890.12

l1_penalty = 3191177428.24: non-zero coefficients = 10: rss= 1008477167020094.38

l1_penalty = 3234142625.56: non-zero coefficients = 10: rss= 1018298780553819.38

l1_penalty = 3277107822.88: non-zero coefficients = 10: rss= 1028247992205977.00

l1_penalty = 3320073020.20: non-zero coefficients = 8: rss= 1034616909232827.38

l1_penalty = 3363038217.52: non-zero coefficients = 8: rss= 1038554735941040.25

l1_penalty = 3406003414.84: non-zero coefficients = 8: rss= 1043237237871701.25

l1_penalty = 3448968612.16: non-zero coefficients = 7: rss= 1046937488751711.50

l1_penalty = 3491933809.

In [22]:
print "The best model has", (best_model['coefficients']['value'].nnz()), "non zero coefficients, its RSS is on validation", \
min_rss, "for a lambda of", best_l1_penalty
best_model.coefficients.print_rows(18)

The best model has 7 non zero coefficients, its RSS is on validation 1.04693748875e+15 for a lambda of 3448968612.16
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       

***16. QUIZ QUESTIONS 5 and 6***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?

2. What features in this model have non-zero coefficients?

If you are using GraphLab Create, enter your answer in simple decimals without commas (e.g. 1131000000), rounded to nearest millions.  3448000000

If you are using scikit-learn, enter your answer in simple decimals without commas (e.g. 4342), rounded to nearest integer. 156

Which of the following features has non-zero coefficients? (Choose all that apply)
* sqft_living   <<----
* bedrooms_square
* sqft_lot_sqrt  
* bathrooms      <<----
* floors


In [28]:
penalty_rss_model = sorted(penalty_rss_model, key = lambda x : x[1])
l1_penalty_optimum = penalty_rss_model[0][0]
model_optimum = penalty_rss_model[0][2]
print(l1_penalty_optimum)
print(model_optimum.coefficients.print_rows(num_rows=model_optimum.num_coefficients))

3448968612.16
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_baseme

In [29]:
best_model.coefficients[best_model.coefficients['value'] != 0]

name,index,value,stderr
(intercept),None,222253.192544,None
bedrooms,None,661.722717782,None
bathrooms,None,15873.9572593,None
sqft_living,None,32.4102214513,None
sqft_living_sqrt,None,690.114773313,None
grade,None,2899.42026975,None
sqft_above,None,30.0115753022,None
